In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
import math
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
from htmldate import find_date
from util.api import get_lts_list
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re
%matplotlib inline
plt.style.use('classic')

In [2]:
lts_list = get_lts_list()
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/dfs/lts-{lts}/lts-{lts}.df')
    df_list.append(df)

In [6]:
#Este fragmento de codigo procesa el total de descargas de los paquetes presentes en la ultima lts que tenga el proyecto
total_downloads = 0
index = 0
print('----------Inicio calculo de descargas------------')
for idx,row in df_list[-1].iterrows():
    # Make a request
    url = "https://hackage.haskell.org/package/%s" % row["package"]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    downloads = 0
    # Extract head of page
    try:
        downloads = 0
        if index == math.trunc(len(df_list[-1])/4):
            print('----------1/4 del progreso------------')
        elif index == math.trunc(len(df_list[-1])/2):
            print('----------2/4 del progreso------------')
        elif index == math.trunc(3*(len(df_list[-1])/4)):
            print('----------3/4 del progreso------------')
        elif index == math.trunc(len(df_list[-1])-1):
            print('----------4/4 del progreso------------')
        datas = soup.find(id = 'properties')
        downloads = datas.find("th", string="Downloads").parent.select("td")
        downloads = str(downloads[0]).replace("<td>", "").split(" ",1)
        total_downloads += int(downloads[0])
    except:
        print(row["package"])
        print('----------------------')
    index += 1
    # print the result

print("Total de descargas en la lts",total_downloads)

----------Inicio calculo de descargas------------
Total de descargas en la lts 85957416


In [ ]:
#Esto se encarga de obtener la descripcion de los paquetes presente en stackage

url = "https://www.stackage.org/lts-19.11"# Si se desea obtener la data de otra lts se modifica la url
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
# Extract head of page
package_description = []

datas = soup.find('tbody')
columnas = datas.find_all("td")
for idx,dato in enumerate(columnas):
    if idx%2:
        package = str(columnas[idx-1]).split('</a')[0].split('">')[1]
        result = re.search(r"(.*)-.*$", package).group(1)
        description = str(dato).split('</')[0].split('td>')[1]
        package_description.append([result,description])

In [ ]:
columns = ["Paquetes","Descripcion"]
columns.extend(lts_list)

In [ ]:
pkg_with_monad_descr = []
pkg_with_all = []
for pkg,descr in package_description:
    descrLower = descr.lower()
    if "monad" in descrLower:
        pkg_with_monad_descr.append(pkg)
        pkg_with_all.append([pkg,descr])
pkg_with_monads = [[] for i in range(len(lts_list))]
totalPkg = ["Total paquetes","Total de paquetes por LTS"]
for idx,df in enumerate(df_list):
    print('|------------------------------|')
    print(lts_list[idx])
    print("cantidad de veces importado equivalente al 0,1%:",len(df)*0.01)
    totalPkg.append(len(df))
    indiceMonads = 0
    for monad in pkg_with_monad_descr:
        usage = 0
        for jdx,row in df.iterrows():
            if monad  in row['deps']:
                usage += 1
        
        if not df[df['package']==monad].empty:
            pkg_with_all[indiceMonads].append(usage)
            pkg_with_monads[idx].append(df[df['package']==monad]['package'])
            if usage >= len(df)*0.01:
                print(monad,' es importado un total  de: ',usage)
        else:
            pkg_with_all[indiceMonads].append(-1)
        indiceMonads+=1
pkg_with_all.append(totalPkg)


In [ ]:
df_excel = pd.DataFrame(pkg_with_all,columns = columns)
df_excel

In [ ]:
df_excel.to_excel('package_with_monads.xlsx', sheet_name='Total_pkg_monad', index=False)